In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

GIsis = nx.from_edgelist([[1,5],[5,3],[1,2],[2,6],[6,4],[6,7]])
sonda = []
sondaTemp = []

def dfs(G,v,visited): #Retorna a matriz de sondas do MPolka
  #print(v)
  #visited = [False] * (GIsis.number_of_nodes() + 1)
  sondaTemp.append(v)
  visited[v] = True
  for w in G.neighbors(v):
    if len(list(G.neighbors(v))) == 1: #DEADEND
        #print(sondaTemp)
        sonda.append(sondaTemp.copy())
        sondaTemp.clear()
    if not visited[w]:
        dfs(G,w,visited)
  return sonda

print('Digite o nó que enviará a sonda em G: ')
node = int(input())
sonda = dfs(GIsis,node,visited=([False] * (GIsis.number_of_nodes() + 1)))

print('Matriz de Sondas = ',sonda)

#OVERHEAD EM 
overheadDP_mPolka = 0
overheadDP_mPINT = 0
overheadDP_INTClassico = 0
overheadCP_mPolka = 0
overheadCP_mPINT = 0
overheadCP_INTClassico = 0

mPolkaDP_fixoPSonda = 10
mPolkaDP_fixoPNo = 20
mPolkaCP_fixoPSonda = 3
mPolkaCP_fixoPNo = 6

mPINTDP_fixoPSonda = 8
mPINTDP_fixoPNo = 15
mPINTCP_fixoPSonda = 3
mPINTCP_fixoPNo = 5

INTClassicoDP_fixoPSonda = 20
INTClassicoDP_fixoPNo = 40
INTClassicoCP_fixoPSonda = 10
INTClassicoCP_fixoPNo = 20

for i in range(len(sonda)):
    overheadDP_mPolka += mPolkaDP_fixoPSonda + mPolkaDP_fixoPNo*len(sonda[i])
    overheadDP_mPINT += mPINTDP_fixoPSonda + mPINTDP_fixoPNo*len(sonda[i])
    overheadDP_INTClassico += INTClassicoDP_fixoPSonda + INTClassicoDP_fixoPNo*len(sonda[i])

    overheadCP_mPolka += mPolkaCP_fixoPSonda + mPolkaCP_fixoPNo*len(sonda[i])
    overheadCP_mPINT += mPINTCP_fixoPSonda + mPINTCP_fixoPNo*len(sonda[i])
    overheadCP_INTClassico += INTClassicoCP_fixoPSonda + INTClassicoCP_fixoPNo*len(sonda[i])

print('Overhead DataPlane MPolka = ',overheadDP_mPolka)
print('Overhead DataPlane MPINT = ',overheadDP_mPINT)
print('Overhead DataPlane INT Clássico = ',overheadDP_INTClassico)

print('Overhead ControlPlane MPolka = ',overheadCP_mPolka)
print('Overhead ControlPlane MPINT = ',overheadCP_mPINT)
print('Overhead ControlPlane INT Clássico = ',overheadCP_INTClassico)

matrixOverhead = {}
matrixOverhead['Topology'] = ['Isis','Isis']
matrixOverhead['Where'] = ['DataPlane','ControlPlane']
matrixOverhead['MPolka'] = [overheadDP_mPolka,overheadCP_mPolka]
matrixOverhead['MPINT'] = [overheadDP_mPINT,overheadCP_mPINT]
matrixOverhead['INT Clássico'] = [overheadDP_INTClassico,overheadCP_INTClassico]
#df = pd.DataFrame(matrixOverhead)
d = {'DataPlane MPolka': [overheadDP_mPolka], 'DataPlane MPINT': [overheadDP_mPINT], 'DataPlane INT Clássico': [overheadDP_INTClassico]}
df = pd.DataFrame(matrixOverhead)
print(df)
#sns.displot(df, x="MPolka",hue="Where",element="step");
#sns.catplot(x="MPolka", y="MPolka", hue="Where",kind="bar", data=df, errorbar = None);

Digite o nó que enviará a sonda em G: 
Matriz de Sondas =  [[1, 5, 3], [2, 6, 4], [7]]
Overhead DataPlane MPolka =  170
Overhead DataPlane MPINT =  129
Overhead DataPlane INT Clássico =  340
Overhead ControlPlane MPolka =  51
Overhead ControlPlane MPINT =  44
Overhead ControlPlane INT Clássico =  170
  Topology         Where  MPolka  MPINT  INT Clássico
0     Isis     DataPlane     170    129           340
1     Isis  ControlPlane      51     44           170
